In [1]:
pip install yt-dlp

   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 3.2/3.2 MB 96.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from __future__ import annotations
import sys
import os
from pathlib import Path
from typing import List
from yt_dlp import YoutubeDL


# ---------- 설정 ---------- #
MAX_HEIGHT = 1080
try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    print("정보: '__file__' 변수를 찾을 수 없습니다. 현재 작업 디렉토리를 기준으로 'output' 폴더를 생성합니다.")
    BASE_DIR = Path.cwd()

OUTPUT_DIR = BASE_DIR / "output"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# ========== 여기에 유튜브 링크 붙여넣기 ========== #
PASTE_YOUTUBE_LINKS_HERE: List[str] = [
    "https://www.youtube.com/watch?v=GoFMr11FFtM"
]


# 다운로드 진행 상황과 완료 안내를 실시간으로 보여주는 콜백 함수
def progress_hook(d):
    if d.get("status") == "downloading":
        p = (d.get("_percent_str") or "").strip()
        spd = (d.get("_speed_str") or "")
        eta = (d.get("_eta_str") or "")
        name = os.path.basename(d.get("filename") or d.get("info_dict", {}).get("_filename") or "")
        sys.stdout.write(f"\r[downloading] {p} {spd} ETA {eta} -> {name}")
        sys.stdout.flush()
    elif d.get("status") == "finished":
        print("\n[post] 다운로드 완료, 병합/후처리 진행 중이니 잠시만 기다려주세요!!")


# MP4 파일 하나를 내려받는 함수
def download_one(url: str) -> None:
    output_template = str(OUTPUT_DIR / "%(title)s.%(ext)s")

    # 1) H.264(AVC)+AAC 조합이 있으면 '복사' 병합 시도 (YouTube 전용 최적화)
    if 'youtube.com' in url or 'youtu.be' in url:
        
        fmt_copy = (
            f"bv*[height<={MAX_HEIGHT}][ext=mp4][vcodec~='^(avc1|h264)']"
            f"+ba[ext=m4a][acodec~='^(mp4a|aac)']"
            f"/b[height<={MAX_HEIGHT}][ext=mp4][vcodec~='^(avc1|h264)'][acodec~='^(mp4a|aac)']"
        )
        
        # 설정
        ydl_copy = {
            "outtmpl": output_template,
            "format": fmt_copy,
            "merge_output_format": "mp4",
            "noplaylist": False,
            "progress_hooks": [progress_hook],
            "windowsfilenames": True,
            "prefer_ffmpeg": True,
            "retries": 10,
            "fragment_retries": 10,
            "postprocessor_args": [
                "-c:v", "copy",
                "-c:a", "copy",
                "-movflags", "+faststart",
            ],
        }

        try:
            with YoutubeDL(ydl_copy) as ydl:
                ydl.download([url])
            return
        except Exception as e:
            print(f"\n[fallback] YouTube H.264(AVC)+AAC 조합(최적화)을 찾지 못했습니다. 범용 다운로더(재인코딩)로 재시도합니다... ({e})")

    # 2) 범용 다운로더 (Naver TV, 기타 사이트, 또는 YouTube 최적화 실패 시)
    #    (Opus를 AAC로 재인코딩하는 블록)
    fmt_any = f"bv*[height<={MAX_HEIGHT}]+ba/b[height<={MAX_HEIGHT}]"
    ydl_transcode = {
        "outtmpl": output_template,
        "format": fmt_any,
        "merge_output_format": "mp4",
        "noplaylist": False,
        "progress_hooks": [progress_hook],
        "windowsfilenames": True,
        "prefer_ffmpeg": True,
        "retries": 10,
        "fragment_retries": 10,
        "concurrent_fragments": 10,
        
        # Opus를 AAC로, VP9을 H.264로 재인코딩하는 설정
        "postprocessor_args": [
            "-c:v", "libx264",
            "-preset", "medium",
            "-pix_fmt", "yuv420p",
            "-c:a", "aac",
            "-b:a", "192k",
            "-movflags", "+faststart"
        ],
    }

    with YoutubeDL(ydl_transcode) as ydl:
        ydl.download([url])

# 실행 인자, 상수, 사용자 입력 순으로 유튜브 링크 목록을 얻기
def parse_urls(argv: List[str]) -> List[str]:
    # 1. 코드 내의 상수 리스트를 최우선으로 확인
    if PASTE_YOUTUBE_LINKS_HERE:
        urls = [u.strip() for u in PASTE_YOUTUBE_LINKS_HERE if u and u.strip()]
        if urls:
            print("정보: 코드 내 'PASTE_YOUTUBE_LINKS_HERE' 리스트에서 URL을 사용합니다.")
            return urls

    # 2. 명령줄 인자(argv) 확인
    if len(argv) > 1:
        urls = [arg for arg in argv[1:] if arg.startswith('http')]
        if urls:
            print("정보: 명령줄 인자에서 URL을 감지했습니다.")
            return urls

    # 3. 위 두 가지가 모두 비어있으면 사용자에게 직접 입력받기
    try:
        url = input("YouTube URL을 입력하세요 (여러 개는 공백으로 구분): ").strip()
    except EOFError:
        url = ""
    return [u for u in url.split() if u]


def main():
    urls = parse_urls(sys.argv)
    if not urls:
        print("URL이 없습니다. 예) python youtub.py https://www.youtube.com/watch?v=...")
        sys.exit(1)

    # .resolve()로 전체 경로를 확인하고 str()로 변환하여 출력
    print(f"출력 폴더: {str(OUTPUT_DIR.resolve())}")
    for u in urls:
        print(f"\n=== 다운로드 시작: {u} ===")
        try:
            download_one(u)
            print(f"\n✓ 저장 완료 (MP4: 최대 {MAX_HEIGHT}p) -> {str(OUTPUT_DIR.resolve())}")
        except Exception as e:
            import traceback
            print(f"\n✗ 실패: {u}\n  이유: {e}")
            print(traceback.format_exc())


if __name__ == "__main__":
    main()

정보: '__file__' 변수를 찾을 수 없습니다. 현재 작업 디렉토리를 기준으로 'output' 폴더를 생성합니다.
정보: 코드 내 'PASTE_YOUTUBE_LINKS_HERE' 리스트에서 URL을 사용합니다.
출력 폴더: C:\Users\Playdata\Downloads\temp\output

=== 다운로드 시작: https://www.youtube.com/watch?v=GoFMr11FFtM ===
[youtube] Extracting URL: https://www.youtube.com/watch?v=GoFMr11FFtM
[youtube] GoFMr11FFtM: Downloading webpage
[youtube] GoFMr11FFtM: Downloading android sdkless player API JSON
[youtube] GoFMr11FFtM: Downloading tv client config
[youtube] GoFMr11FFtM: Downloading tv player API JSON
[youtube] GoFMr11FFtM: Downloading web safari player API JSON
[youtube] GoFMr11FFtM: Downloading player 7d647a07-main


         player = https://www.youtube.com/s/player/7d647a07/player_ias.vflset/en_US/base.js
         n = PWdRnbRyTfdpQslk ; player = https://www.youtube.com/s/player/7d647a07/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


[youtube] GoFMr11FFtM: Downloading m3u8 information
[info] GoFMr11FFtM: Downloading 1 format(s): 137+140
[download] Sleeping 4.00 seconds as required by the site...
[download] Destination: c:\Users\Playdata\Downloads\temp\output\얼떨결에 연봉 공개한 최강야구 정용검 캐스터 (20...억!？？？？？ㄷㄷ).f137.mp4
[download] 100% of  335.77MiB in 00:00:07 at 46.37MiB/s  한 최강야구 정용검 캐스터 (20...억!？？？？？ㄷㄷ).f137.mp44
[post] 다운로드 완료, 병합/후처리 진행 중이니 잠시만 기다려주세요!!

[download] Destination: c:\Users\Playdata\Downloads\temp\output\얼떨결에 연봉 공개한 최강야구 정용검 캐스터 (20...억!？？？？？ㄷㄷ).f140.m4a
[download] 100% of   33.51MiB in 00:00:07 at 4.74MiB/s   한 최강야구 정용검 캐스터 (20...억!？？？？？ㄷㄷ).f140.m4aa
[post] 다운로드 완료, 병합/후처리 진행 중이니 잠시만 기다려주세요!!

[Merger] Merging formats into "c:\Users\Playdata\Downloads\temp\output\얼떨결에 연봉 공개한 최강야구 정용검 캐스터 (20...억!？？？？？ㄷㄷ).mp4"
Deleting original file c:\Users\Playdata\Downloads\temp\output\얼떨결에 연봉 공개한 최강야구 정용검 캐스터 (20...억!？？？？？ㄷㄷ).f137.mp4 (pass -k to keep)
Deleting original file c:\Users\Playdata\Downloads\temp\output\얼떨결에 연